#### TODO
### SKIP GRAM


In [173]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, FreqDist
import string
from tqdm import tqdm
puncts = string.punctuation
import torch
from torch import nn
import torch.nn.functional as F
import math

In [116]:
lemmatizer = WordNetLemmatizer()
def context_ngrams(word_arr, context_size, mode="uni_dir"):
    txt = []
    end_point = len(word_arr) - context_size if mode=="bi_dir" else len(word_arr)
    for i in range(context_size, end_point):
        if mode=="uni_dir":
            txt.append([word_arr[i-context_size: i], word_arr[i]])
        else:
            txt.append([word_arr[i-context_size: i] +  word_arr[i+1: i + context_size+1], 
                        word_arr[i]])
    return txt

def preprocess(input_corpus:list):
    string_without_puct = ["".join([char for char in vec if char not in puncts]) for vec in input_corpus]

    pos_sample_dict = {}

    for line in string_without_puct:
        context1 = context_ngrams(word_tokenize(line), 2, mode="bi_dir")
        if context1:
            for context, target in context1:
                if target not in pos_sample_dict.keys():
                   pos_sample_dict[target] = []
                pos_sample_dict[target] += [c_word for c_word in context if c_word!=target]

    return string_without_puct, pos_sample_dict

def sort_and_select(pos_dict, cutoff_max=50):
    final_dict = {}
    for k, values in pos_dict.items():
        val_dict = {}
        for word in values:
            if word not in val_dict.keys():
                val_dict[word] = 0
            val_dict[word] += 1
        val_tup_list = list(val_dict.items())
        val_tup_sorted =  sorted(val_tup_list, key=lambda x:x[1], reverse=True)
        final_dict[k] = list(map(lambda x: x[0], val_tup_sorted))[:cutoff_max]
    return final_dict

In [117]:
df =  pd.read_csv('/Users/gauneg/datasets/fun_dsets/sourth_park_ds.csv')
df_cartman = df[df['Character']=='Cartman']
dset_strings = df_cartman['Line'].str.lower().str.strip().values
pros_dset, pos_dict = preprocess(dset_strings)
# pos_training_set = sort_and_select(pos_dict)


### Discarding word with the following probability (To be done)

$$P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}}$$

t: threshold (~$10^{-5}$) , $w_i$: word

In [230]:
vocab_set = [word for sent in pros_dset for word in word_tokenize(sent)]

word_to_ix = {w : i for i, w in enumerate(vocab_set)}
ix_to_word = {i:w for w, i in word_to_ix.items()}

def apply_threshold(all_words, t=0.01):
    freq_dict = dict(FreqDist(all_words))
    prob_dist = {w:(1 - math.sqrt(t/f)) for w,f in freq_dict.items()}
    return prob_dist

def apply_tfidf(dataset):
    tokenized_sents = [word_tokenize(data) for data in dataset]
    freq_dict = dict(FreqDist([word for tok_sent in tokenized_sents for word in tok_sent]))
    N = sum(freq_dict.values())
    tf_dict = {w:f for w,f in freq_dict.items()}
    word_doc_occur = {}
    for word, w_freq in tf_dict.items():
        word_doc_val = 0
        for tok_sent in tokenized_sents:
            if word in tok_sent:
                word_doc_val += 1
        word_doc_occur[word] = (w_freq, math.log(len(tokenized_sents)/word_doc_val))
    return word_doc_occur



# prob_dict = apply_threshold(vocab_all)
# sorted(list(prob_dict.items()), key=lambda x: x[1], reverse=True)    

In [231]:
dict_all_x = apply_tfidf(pros_dset)


In [244]:
sorted(list(dict_all_x.items()), key=lambda x: (math.pow(x[1][0], 1/3))*x[1][1], reverse=True)[-20:]

[('worldwide', (1, 9.1874810778301)),
 ('extermination', (1, 9.1874810778301)),
 ('ddont', (1, 9.1874810778301)),
 ('ss', (1, 9.1874810778301)),
 ('thathat', (1, 9.1874810778301)),
 ('unrelated', (1, 9.1874810778301)),
 ('throgh', (1, 9.1874810778301)),
 ('aj', (1, 9.1874810778301)),
 ('feeley', (1, 9.1874810778301)),
 ('funplex', (1, 9.1874810778301)),
 ('physics', (1, 9.1874810778301)),
 ('yeehehehes', (1, 9.1874810778301)),
 ('230', (1, 9.1874810778301)),
 ('splashed', (1, 9.1874810778301)),
 ('stingray', (1, 9.1874810778301)),
 ('kya', (1, 9.1874810778301)),
 ('bunsow', (1, 9.1874810778301)),
 ('nahuh', (1, 9.1874810778301)),
 ('raku', (1, 9.1874810778301)),
 ('racksa', (1, 9.1874810778301))]

In [184]:
def create_negatives_sample(pos_dict, vocab):
    negative_dict = {}
    for k,v in tqdm(pos_dict.items()):
        scan_for_word = None
        curr_neg = []
        for word in vocab:
            scan_for_word = word
            for word_pos in v:
                if scan_for_word==word_pos or scan_for_word==k:
                    scan_for_word = None
            if scan_for_word:
                curr_neg.append(scan_for_word)
        negative_dict[k] = curr_neg
    return negative_dict

In [185]:
neg_list = create_negatives_sample(pos_dict, vocab_set)

100%|██████████| 7635/7635 [01:31<00:00, 83.69it/s] 


In [187]:
pos_data_dict = sort_and_select(pos_dict, cutoff_max=5)
neg_data_dict = sort_and_select(neg_list, cutoff_max=5)

{'na': ['gon', 'wan', 'im', 'be', 'you'],
 'miss': ['na', 'gon', 'you', 'terrance', 'phillip'],
 'him': ['to', 'i', 'you', 'a', 'in'],
 'im': ['gon', 'na', 'not', 'a', 'you'],
 'gon': ['na', 'im', 'be', 'were', 'youre'],
 'chef': ['just', 'its', 'and', 'did', 'a'],
 'and': ['the', 'i', 'you', 'a', 'to'],
 'iand': ['chef', 'and', 'i', 'dont'],
 'i': ['you', 'have', 'and', 'to', 'know'],
 'dont': ['i', 'you', 'know', 'have', 'we'],
 'know': ['you', 'i', 'what', 'dont', 'that'],
 'how': ['you', 'i', 'know', 'to', 'about'],
 'to': ['the', 'have', 'you', 'i', 'be'],
 'did': ['you', 'i', 'what', 'it', 'the'],
 'seem': ['to', 'you', 'doesnt', 'have', 'like'],
 'a': ['of', 'to', 'i', 'and', 'is'],
 'little': ['a', 'in', 'of', 'and', 'just'],
 'uh': ['i', 'you', 'oh', 'and', 'the'],
 'trippy': ['little', 'uh', 'to', 'you'],
 'oh': ['god', 'my', 'you', 'i', 'no'],
 'boy': ['a', 'the', 'little', 'band', 'and'],
 'cant': ['i', 'you', 'just', 'be', 'it'],
 'wait': ['for', 'i', 'a', 'to', 'no'],
 'h

In [ ]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim) -> None:
        super(Word2Vec, self).__init__()
        self.target_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.context_embedding = nn.Embedding(vocab_size, embedding_dim)
    
    def forward(self, pos_target, pos_context, neg_target, neg_context):
        """
            u-> 1, v-> 0
            pos_u: list of center word ids for positive word pairs.
            pos_v: list of neibor word ids for positive word pairs.
            neg_u: list of center word ids for negative word pairs.
            neg_v: list of neibor word ids for negative word pairs.
        """
        tar_pos = self.target_embedding(pos_target) 
        con_pos = self.context_embedding(pos_context)
        pos_score = torch.mul(tar_pos, con_pos)
        pos_score = torch.sum(pos_score, dim=1)
        pos_score = F.logsigmoid(pos_score)
        tar_neg = self.target_embedding(neg_target)
        con_neg = self.context_embedding(neg_context)
        # TODO: When